In [85]:
import openpyxl as px
import numpy as np
import requests
import time
from datetime import date

In [93]:
def get_unique_elements(lst):
    unique_lst = []
    
    for item in lst:
        # Check if the item is already in the unique list
        if item not in unique_lst:
            unique_lst.append(item)
    
    return unique_lst

In [44]:
solved_url_all=["https://codeforces.com/problemset/status/1832/problem/C","https://codeforces.com/problemset/status/1826/problem/A","https://codeforces.com/problemset/status/1822/problem/D"]
solved_url="https://codeforces.com/problemset/status/1832/problem/C"
solved_url_test="https://codeforces.com/problemset/status/1832/problem/C"

In [45]:
# Global constants initialised
#region
global x_profile1,x_profile2,x_profile3,x_profile4,x_time,x_verdict,x_submission2,end_part_url,x_timetaken,domain,row,batch_pages
global element_index_problem_id,element_index_problem_name,element_index_problem_url,element_index_tags,element_index_ratings,element_index_solved_numbers,element_index_solved_url
global profile_index_id,profile_index_url,profile_index_submission,profile_index_submission_url,profile_index_submission_time,profile_index_timetaken,profile_index_verdict
global friend_id_map,headers,request_error
request_error="503 Service Temporarily Unavailable"
headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'}
friend_id_map={}# can write a map of your friend name and its cf_id
x_profile1='<a href="/profile/'
x_profile2='class="rated-user'
x_profile3="<small>*</small>"
x_profile4='href="/team'
x_time='<span class="format-time"'
x_verdict='<span class="submissionVerdictWrapper"'
x_submission1="submissionId="
x_submission2='<a class="view-source"'
end_part_url="?order=BY_PROGRAM_LENGTH_ASC"
x_timetaken='class="time-consumed-cell"'
domain="https://codeforces.com"
frnd_adder="?friends=on"
row=[]
pages_response=[]
element_index_problem_id=0
element_index_problem_name=1
element_index_problem_url=2
element_index_tags=3
element_index_rating=4
element_index_solved_number=5
element_index_solved_url=6
profile_index_id=7
profile_index_url=8
profile_index_submission=9
profile_index_submission_url=10
profile_index_submission_time=11
profile_index_timetaken=12
profile_index_verdict=13
# """
# https://sqqihao.github.io/trillworks.html
# """
#endregion

In [46]:
#A sample of all data extracted for referring indices
#region
# ['1832C',
#    'Contrast Value',
#    'https://codeforces.com/problemset/problem/1832/C',
#    'greedy, implementation',
#    1200,
#    12458,
#    'https://codeforces.com/problemset/status/1832/problem/C',
#    'friend_url',
#    'friend_id',
#    submission_id,
#    'submission_url',
#    'time_of_submission',
#    'execution_time',
#    'Accepted']
#endregion

In [48]:
cookies = {} # See the Readme file

In [49]:
# To return the page numbers for a particular problem
def url_extract_pages(url):    
    res=requests.get(url)
    page_data=res.text.split('\n')
    page_size=len(page_data)
    for i in range(page_size):
        if '</ul>' in page_data[i]:
            endpage_line=i-3
    pages=int(page_data[endpage_line].split('>')[2][:-3])
    del page_data, page_size, res
    return pages

In [8]:
#Async extractor functions
#region
# async def fetch_url(session, url):
#     async with session.get(url,headers=headers) as response:
#         iserror=True
#         while(iserror):
#             response_given=await response.text()
#             if request_error in response_given:
#                 continue
#             else:
#                 print(url," is done")
#                 return response_given

# async def scrape_urls(urls):
#     async with aiohttp.ClientSession() as session:
#         tasks = []
#         for url in urls:
#             task = asyncio.ensure_future(fetch_url(session, url))
#             tasks.append(task)
#         responses = await asyncio.gather(*tasks)
#         return responses

# async def extracter(urls):
#     # page_lis=[page for page in range(1,pages+1)]
#     responses = await scrape_urls(urls)
#     # all_responses=[]
#     # async for response in responses:
#     #     all_responses.append(response)
#     # all_responses=list(dict(zip(page_lis,responses)).values())
#     # for url, response in all_responses:
#         # print(f'Response from {url}:\n{response}\n')
#     return responses
    # return all_responses

# async def response_lis_maker(coroutines,all_responses):
#     # all_responses=[]
#     async for response in coroutines:
#         all_responses.append(response)
    # return all_responses

# async def response_lis_maker(coroutines):
#     result=await response_lis_initiate(coroutines)
#     return result

# Run the main function
# start=time.time()
# all_response = await extracter(links)
# end=time.time()
# print("Total time taken = ",end-start)
#endregion

In [50]:
# Getting all the problem data for each problems
def problemdata_by_rating(rating):
    problemdata=[]
    wb=px.load_workbook(f'problem by ratings/{rating}/{rating}.xlsx')
    i=1
    while(i!=0):
        sheet_name=f'{rating}_pg{i}'
        if sheet_name in wb.sheetnames :
            wb.active=wb[sheet_name]
            sheet=wb.active
            j=0
            while(not(sheet.cell(row=2+j,column=1).value == None)):
                problemsample=[]
                flag=1
                # Problem ID = 0
                problemsample.append(sheet.cell(row=2+j,column=1).value.split('"')[3])
                # Problem Name = 1
                try:
                    problemsample.append(sheet.cell(row=2+j,column=2).value.split('"')[3])
                except:
                    flag=0
                    # print("i = ",i," | j = ",j)
                # Problem URL = 2
                problemsample.append(sheet.cell(row=2+j,column=2).value[12:].split('"')[0])
                # Problem Tags = 3
                problemsample.append(sheet.cell(row=2+j,column=3).value)
                # Problem Rating = 4
                problemsample.append(sheet.cell(row=2+j,column=4).value)
                # Problem Solved numbers = 5
                problemsample.append(int(sheet.cell(row=2+j,column=5).value.split(',')[1][1:-1]))
                # Problem Solved URL = 6
                problemsample.append(sheet.cell(row=2+j,column=5).value[12:].split('"')[0])
                j+=1
                if(flag):
                    problemdata.append(problemsample)
            i+=1
        else:
            i=0
    del wb,i,sheet_name,sheet,problemsample
    return problemdata


In [78]:
def find_frndwise(problemdata):
    row_rating=[]
    for problem in problemdata:
        frndlist_link=problem[element_index_solved_url]+frnd_adder
        response=requests.get(frndlist_link,cookies=cookies).text
        problem_row=[]
        try:
            for friend in friend_id_map:
                if friend_id_map[friend] in response:
                    # print("We found : ",friend_id_map[friend])
                    flag=1
                    data=response.split('\n')
                    size=len(data)
                    
                    # #### Data to be extracted

                    # ### Profile Name, ID, URL
                    profile_lines=[]
                    for i in range(size):
                        if x_profile1 in data[i] and x_profile2 in data[i]:
                            profile_lines.append(i)
                    profiles=[(data[i].split('>')[3][:-3] if (x_profile3 in data[i]) else (friend_id_map[friend] if x_profile4 in data[i] else data[i].split('>')[1][:-3])) for i in profile_lines]
                    profile_url=[domain+'/profile/'+i for i in profiles]
                    
                    # Finding the cell location of the profile
                    pos=profiles.index(friend_id_map[friend])
                    
                    frnd_profile=profiles[pos]
                    frnd_profile_url=profile_url[pos]
                    # print("We found : ",frnd_profile)

                    # ### Submission IDs
                    sub_lines=[]
                    for i in range(size):
                        if x_submission1 in data[i] and x_submission2 in data[i]:
                            sub_lines.append(i)
                    frnd_submission_id=int(data[sub_lines[pos]].split('>')[1][:-3])
                    frnd_submission_url=domain+data[sub_lines[pos]].split('"')[5]
                    # submission_ids=[int(data[i].split('>')[1][:-3]) for i in sub_lines]
                    # submission_url=[ domain+data[i].split('"')[5] for i in sub_lines]

                    # ### Time of submissions
                    time_lines=[]
                    for i in range(size):
                        if x_time in data[i]:
                            time_lines.append(i)
                    frnd_time_of_sub=data[time_lines[pos]].split('>')[1][:-6]
                    # time_of_subs=[data[i].split('>')[1][:-6] for i in time_lines]
                    
                    timetaken_lines=[]
                    for i in range(size):
                        if x_timetaken in data[i] :
                            timetaken_lines.append(i+1)
                    frnd_timetaken=data[timetaken_lines[pos]].strip().split('&')[0]+data[timetaken_lines[pos]].strip().split('&')[1][-2:]
                    # timetaken_lines=[data[i][4:-13].strip()+data[i][-7:-5] for i in timetaken_lines]
                    
                    # ### Verdicts
                    verdict_lines=[]
                    for i in range(size):
                        if x_verdict in data[i]:
                            verdict_lines.append(i)
                    frnd_verdict=data[verdict_lines[pos]].split('>')[2][:-6]
                    # verdicts=[data[i].split('>')[2][:-6] for i in verdict_lines]
                    element=problem+[frnd_profile,frnd_profile_url,frnd_submission_id,frnd_submission_url,frnd_time_of_sub,frnd_timetaken,frnd_verdict]
                    problem_row.append(element)
                if not(problem_row==[]):
                    row_rating.append(problem_row)
        except:
            print("Some Error Occured")
            print("Rating: ",problem[element_index_rating])
            print("Problem URL: ",problem[element_index_problem_url])
            print("Solved URL: ",problem[element_index_solved_url])
            return response
            break
    return row_rating

In [79]:
# problemdata=problemdata_by_rating(1300)
# test_data=requests.get(problemdata[0][element_index_solved_url]+frnd_adder,cookies=cookies).text.split("\n")
# for i,line in enumerate(test_data):
#     print(i,line)
# x_profile3 in test_data[1712]
# i=1712
# (test_data[i].split('>')[3][:-3] if (x_profile3 in test_data[i]) else ("*" if x_profile4 in test_data[i] else test_data[i].split('>')[1][:-3]))

In [80]:
def tabulate_frnddata(frnd_data,rating):
    
    ### Working with the excel sheets  
    save_path=f"{rating}.xlsx"
    sheet_name=f"{rating} {str(date.today())}"
    # print("Sheet name used is : ",sheet_name)
    try:
        wb=px.load_workbook(save_path)
        if sheet_name in wb.sheetnames:
            pass
        else:
            wb.create_sheet(sheet_name)    
    except FileNotFoundError:
        wb=px.Workbook()
        wb.create_sheet(sheet_name)
        del wb['Sheet']                 
    wb.active=wb[sheet_name]
    sheet=wb.active
    sheet["A1"] = "ID"
    sheet["B1"] = "Name"
    sheet["C1"] = "Tags"
    sheet["D1"] = "Rating"
    sheet["E1"] = "Solved numbers"
    sheet["F1"] = "Profile ID"
    sheet["G1"] = "Submission ID"
    sheet["H1"] = "Submission Time"
    sheet["I1"] = "Execution Time"
    sheet["J1"] = "Verdict"
    
    ### Loading and formatting the data in excel
    i=0
    for group_sample in frnd_data:
        for sample in group_sample:
            #Problem ID
            sheet.cell(row=2+i, column=1).value='=HYPERLINK("{}", "{}")'.format(sample[element_index_problem_url],sample[element_index_problem_id])
            sheet.cell(row=2+i, column=1).font=px.styles.Font(color="0000FF")
            #Problem Name
            sheet.cell(row=2+i, column=2).value='=HYPERLINK("{}", "{}")'.format(sample[element_index_problem_url],sample[element_index_problem_name])
            sheet.cell(row=2+i, column=2).font=px.styles.Font(color="0000FF")
            #Problem Tags
            sheet.cell(row=2+i, column=3).value=sample[element_index_tags]
            #Problem Rating
            sheet.cell(row=2+i, column=4).value=sample[element_index_rating]
            #Problem Solved Numbers
            sheet.cell(row=2+i, column=5).value='=HYPERLINK("{}", {})'.format(sample[element_index_solved_url],sample[element_index_solved_number])
            sheet.cell(row=2+i, column=5).font=px.styles.Font(color="0000FF")
            
            #Profiles
            sheet.cell(row=2+i, column=6).value='=HYPERLINK("{}", "{}")'.format(sample[profile_index_url],sample[profile_index_id])
            sheet.cell(row=2+i, column=6).font=px.styles.Font(color="0000FF")
            #Submissions
            sheet.cell(row=2+i, column=7).value='=HYPERLINK("{}", "{}")'.format(sample[profile_index_submission_url],sample[profile_index_submission])
            sheet.cell(row=2+i, column=7).font=px.styles.Font(color="0000FF")
            #Submission Time
            sheet.cell(row=2+i, column=8).value=sample[profile_index_submission_time]
            #Execution Time
            sheet.cell(row=2+i, column=9).value=sample[profile_index_timetaken]
            #Verdict
            sheet.cell(row=2+i, column=10).value=sample[profile_index_verdict]
            i+=1

    #### Final saving
    try:  
        wb.save(save_path)
        print("File ",save_path," was successful.")
    except Exception as e:
        os.mkdir(dir) 
        wb.save(save_path)
        print("File ",save_path," was successful. but directory : ",dir," didn't exist so we created it.")

In [97]:
### Main function for extraction

def friend_problem_extract(ratings):
    if isinstance(ratings,list):
        for rating in ratings:
            t1=time.time()
            problemdata=problemdata_by_rating(rating)
            t2=time.time()
            print(f"Rating: {rating} | Listing Time: {t2-t1:.2f} ",end=" | ")
            found_data=get_unique_elements(find_frndwise(problemdata))
            t3=time.time()
            print(f"Finding Time: {t3-t2:.2f} ",end=" | ")
            tabulate_frnddata(found_data,rating)
            t4=time.time()
            print(f"Tabulating Time: {t4-t3:.2f} ")
            print(f"Total time taken for {rating} = {t4-t1:.3f}")
    else:
        rating=ratings
        t1=time.time()
        problemdata=problemdata_by_rating(rating)
        t2=time.time()
        print(f"Rating: {rating} | Listing Time: {t2-t1:.2f} ",end=" | ")
        found_data=get_unique_elements(find_frndwise(problemdata))
        t3=time.time()
        print(f"Finding Time: {t3-t2:.2f} ",end=" | ")
        tabulate_frnddata(found_data,rating)
        t4=time.time()
        print(f"Tabulating Time: {t4-t3:.2f} ")
        print(f"Total time taken for {rating} = {t4-t1:.3f}")

In [98]:
friend_problem_extract([1200,1300,1400,1500,1600,1700,1800,1900,2000,2100,2200])

Rating: 1200 | Listing Time: 0.26  | Finding Time: 308.65  | File  1200.xlsx  was successful.
Tabulating Time: 0.08 
Total time taken for 1200 = 308.992
Rating: 1300 | Listing Time: 0.07  | Finding Time: 343.07  | File  1300.xlsx  was successful.
Tabulating Time: 0.07 
Total time taken for 1300 = 343.202
Rating: 1400 | Listing Time: 0.06  | Finding Time: 300.45  | File  1400.xlsx  was successful.
Tabulating Time: 0.07 
Total time taken for 1400 = 300.580
Rating: 1500 | Listing Time: 0.02  | Finding Time: 353.52  | File  1500.xlsx  was successful.
Tabulating Time: 0.06 
Total time taken for 1500 = 353.602
Rating: 1600 | Listing Time: 0.11  | Finding Time: 376.43  | File  1600.xlsx  was successful.
Tabulating Time: 0.05 
Total time taken for 1600 = 376.588
Rating: 1700 | Listing Time: 0.03  | Finding Time: 346.69  | File  1700.xlsx  was successful.
Tabulating Time: 0.08 
Total time taken for 1700 = 346.801
Rating: 1800 | Listing Time: 0.07  | Finding Time: 326.07  | File  1800.xlsx  was 